### Exclude atoms farthur than 8A From Ligand

In [1]:
positivePDBDir = "../positive_truncated_data/"
positiveTarDir = "../positive_graph_save8A/"
negativePDBDir = "../negative_pdb/"
negativeTarDir = "../negative_graph_save8A/"

In [2]:
valid_elements = ['N', 'C', 'O', 'S', 'H', 'P', 'F', 'CL',  'BR',  'ZN']
element_onehot = {}
for idx, ele in enumerate(valid_elements):
    element_onehot[ele] = [0]*len(valid_elements)
    element_onehot[ele][idx] = 1
    
element_onehot

{'N': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'C': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 'O': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 'S': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 'H': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 'P': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 'F': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 'CL': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 'BR': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 'ZN': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

In [17]:
def ligandCenter(pdbData):
    minX, minY, minZ = 99999.0, 99999.0, 99999.0
    maxX, maxY, maxZ = -99999.0, -99999.0, -99999.0
    for data in pdbData:
        if data[1] == 0:     # Ignore Protein Atoms
            continue
        minX = data[3] if data[3]<minX else minX
        minY = data[4] if data[4]<minY else minY
        minZ = data[5] if data[5]<minZ else minZ
        maxX = data[3] if data[3]>maxX else maxX
        maxY = data[4] if data[4]>maxY else maxY
        maxZ = data[5] if data[5]>maxZ else maxZ
    return minX, minY, minZ, maxX, maxY, maxZ

In [4]:
def fartherThan8A(ligandBox, atomPosition):
    x, y, z = atomPosition[0], atomPosition[1], atomPosition[2]
    minX, minY, minZ, maxX, maxY, maxZ = ligandBox
#     x_dis = x - minX
#     y_dis = y - minY
#     z_dis = z - minZ
#     disMin = math.sqrt(x_dis**2+y_dis**2+z_dis**2)
    
#     x_dis = x - maxX
#     y_dis = y - maxY
#     z_dis = z - maxZ
#     disMax = math.sqrt(x_dis**2+y_dis**2+z_dis**2)
    centerX, centerY, centerZ = (minX+maxX)/2, (minY+maxY)/2, (minZ+maxZ)/2
    disMin = math.sqrt((centerX-minX)**2+(centerY-minY)**2+(centerZ-minZ)**2)
    disMax = math.sqrt((centerX-maxX)**2+(centerY-maxY)**2+(centerZ-maxZ)**2)
    radius = disMin if disMin > disMax else disMax
    
    dis = math.sqrt((centerX-x)**2+(centerY-y)**2+(centerZ-z)**2)
    return (dis-radius)>8

In [5]:
from dgl.data import DGLDataset
import os
import numpy as np
import pandas as pd
import torch as th
import dgl
import json
import math

Using backend: pytorch


In [19]:
for pdb_name in os.listdir(negativePDBDir):
    pdb_data = []                        #一个pdb文件中的信息汇总
    # [element_name, protein_or_ligand, chain_id, x, y, z]
    with open(negativePDBDir+pdb_name) as pdb_file:
        lines = pdb_file.readlines()
        flag = False
        for i, line in enumerate(lines):
            if (line[:4]=='ATOM' or line[:6]=='HETATM') and line[76:78].strip() in valid_elements:
                atom = ['C', 0, 'A', 0.0, 0.0, 0.0]#0代表蛋白质分子。 1代表ligand分子
                atom[0] = line[76:78].strip()                   #element_name
                atom[1] = 0 if line[:4]=='ATOM' else 1  #protein or ligand
                if (i > 0 and int(line[6:11]) < int(lines[i-1][6:11])) or flag:
                    flag = True
                    atom[1] = 1
                atom[2] = line[21]                      #chain_id
                atom[3] = float(line[30:38].strip())    #x
                atom[4] = float(line[38:46].strip())    #y
                atom[5] = float(line[46:54].strip())    #z
                pdb_data.append(atom)
    ligandBox = ligandCenter(pdb_data)
    graph_edata = [] #edge weights
    u_set = []
    v_set = []
    graph_ndata = []
    nodeDict = {}
    pos = 0
    for i in range(len(pdb_data)):
        if fartherThan8A(ligandBox, [pdb_data[i][3], pdb_data[i][4], pdb_data[i][5]]):
            continue
        graph_ndata.append([element_onehot[pdb_data[i][0]]]) #[[ele1, other_feature], [ele2, other_feature],....]
        nodeDict[i] = pos
        pos += 1
    for i in nodeDict.items():
        for j in nodeDict.items():
            x_dis = pdb_data[i[0]][3] - pdb_data[j[0]][3]
            y_dis = pdb_data[i[0]][4] - pdb_data[j[0]][4]
            z_dis = pdb_data[i[0]][5] - pdb_data[j[0]][5]
            dis = math.sqrt(x_dis**2+y_dis**2+z_dis**2)
            if dis < 2.0 and pdb_data[i[0]][1]==pdb_data[j[0]][1] and pdb_data[i[0]][2] == pdb_data[j[0]][2]:
                graph_edata.append([1])
                graph_edata.append([1])
                u_set.append(i[1])
                v_set.append(j[1])
                u_set.append(j[1])
                v_set.append(i[1])
            elif dis < 5.0 and (pdb_data[i[0]][1]==pdb_data[j[0]][1] or pdb_data[i[0]][2] == pdb_data[j[0]][2]):
                graph_edata.append([dis])
                graph_edata.append([dis])
                u_set.append(i[1])
                v_set.append(j[1])
                u_set.append(j[1])
                v_set.append(i[1])
    g = dgl.DGLGraph()
    g.add_nodes(len(graph_ndata))
    g.add_edges(u_set, v_set)
    print(pdb_name, len(pdb_data), len(graph_edata), len(graph_ndata), len(u_set), len(v_set))
    g.edata['h'] = th.tensor(graph_edata)
    g.ndata['h'] = th.tensor(graph_ndata)

    dgl.save_graphs(negativeTarDir+pdb_name[:-4], [g])

4ufm_ligand_3.pdb 5109 13364 298 13364 13364
3c2o_ligand_5.pdb 2035 9238 229 9238 9238
3nuo_ligand_5.pdb 791 15772 366 15772 15772
6b98_ligand_3.pdb 2755 12062 301 12062 12062
3mho_ligand_2.pdb 2060 29456 606 29456 29456
4myd_ligand_5.pdb 1954 19402 445 19402 19402
6eyt_ligand_3.pdb 2495 24882 531 24882 24882
4aci_ligand_7.pdb 1412 13750 313 13750 13750
3f33_ligand_6.pdb 1367 9380 226 9380 9380
1h46_ligand_2.pdb 19 438 19 438 438
4u54_ligand_5.pdb 3255 11682 263 11682 11682
3bpc_ligand_8.pdb 1705 6842 173 6842 6842
2oxy_ligand_8.pdb 2736 132216 2736 132216 132216
4zwz_ligand_6.pdb 2070 29056 614 29056 29056
3n2p_ligand_1.pdb 2082 33674 717 33674 33674
2a4m_ligand_5.pdb 2559 330 15 330 330
3ryx_ligand_6.pdb 21 502 21 502 502
5mmg_ligand_8.pdb 989 18194 403 18194 18194
2azr_ligand_4.pdb 2443 12468 294 12468 12468
4ara_ligand_5.pdb 4178 27546 633 27546 27546
4umb_ligand_3.pdb 2567 11300 282 11300 11300
4qpl_ligand_4.pdb 1274 21306 473 21306 21306
3f5l_ligand_8.pdb 3832 30584 646 30584 305

KeyboardInterrupt: 

In [18]:
#pdb_name = "4bb9_ligand_2.pdb"
pdb_name = "3p7i_ligand_5.pdb"
pdb_data = []                        #一个pdb文件中的信息汇总
# [element_name, protein_or_ligand, chain_id, x, y, z]
with open(negativePDBDir+pdb_name) as pdb_file:
    lines = pdb_file.readlines()
    flag = False
    for i, line in enumerate(lines):
        if (line[:4]=='ATOM' or line[:6]=='HETATM') and line[76:78].strip() in valid_elements:
            atom = ['C', 0, 'A', 0.0, 0.0, 0.0]#0代表蛋白质分子。 1代表ligand分子
            atom[0] = line[76:78].strip()                   #element_name
            atom[1] = 0 if line[:4]=='ATOM' else 1  #protein or ligand
            if (i > 0 and int(line[6:11]) < int(lines[i-1][6:11])) or flag:
                flag = True
                atom[1] = 1
            atom[2] = line[21]                      #chain_id
            atom[3] = float(line[30:38].strip())    #x
            atom[4] = float(line[38:46].strip())    #y
            atom[5] = float(line[46:54].strip())    #z
            pdb_data.append(atom)
ligandBox = ligandCenter(pdb_data)
print(pdb_data)
print(ligandBox)
graph_edata = [] #edge weights
u_set = []
v_set = []
graph_ndata = []
nodeDict = {}
pos = 0
for i in range(len(pdb_data)):
    if fartherThan8A(ligandBox, [pdb_data[i][3], pdb_data[i][4], pdb_data[i][5]]):
        continue
    graph_ndata.append([element_onehot[pdb_data[i][0]]]) #[[ele1, other_feature], [ele2, other_feature],....]
    nodeDict[i] = pos
    pos += 1
for i in nodeDict.items():
    for j in nodeDict.items():
        x_dis = pdb_data[i[0]][3] - pdb_data[j[0]][3]
        y_dis = pdb_data[i[0]][4] - pdb_data[j[0]][4]
        z_dis = pdb_data[i[0]][5] - pdb_data[j[0]][5]
        dis = math.sqrt(x_dis**2+y_dis**2+z_dis**2)
        if dis < 2.0 and pdb_data[i[0]][1]==pdb_data[j[0]][1] and pdb_data[i[0]][2] == pdb_data[j[0]][2]:
            graph_edata.append([1])
            graph_edata.append([1])
            u_set.append(i[1])
            v_set.append(j[1])
            u_set.append(j[1])
            v_set.append(i[1])
        elif dis < 5.0 and (pdb_data[i[0]][1]==pdb_data[j[0]][1] or pdb_data[i[0]][2] == pdb_data[j[0]][2]):
            graph_edata.append([dis])
            graph_edata.append([dis])
            u_set.append(i[1])
            v_set.append(j[1])
            u_set.append(j[1])
            v_set.append(i[1])
g = dgl.DGLGraph()
g.add_nodes(len(graph_ndata))
g.add_edges(u_set, v_set)
print(pdb_name, len(pdb_data), len(graph_edata), len(graph_ndata), len(u_set), len(v_set))
g.edata['h'] = th.tensor(graph_edata)
g.ndata['h'] = th.tensor(graph_ndata)

dgl.save_graphs(negativeTarDir+pdb_name[:-4], [g])

[['N', 0, 'A', 20.585, -19.231, 27.657], ['C', 0, 'A', 21.309, -19.411, 26.372], ['C', 0, 'A', 20.405, -19.978, 25.254], ['O', 0, 'A', 20.074, -19.219, 24.319], ['C', 0, 'A', 22.507, -20.348, 26.585], ['C', 0, 'A', 23.441, -20.472, 25.369], ['C', 0, 'A', 24.913, -20.457, 25.755], ['C', 0, 'A', 25.428, -19.048, 26.024], ['N', 0, 'A', 26.896, -19.062, 25.872], ['N', 0, 'A', 20.016, -21.258, 25.371], ['C', 0, 'A', 19.13, -21.939, 24.402], ['C', 0, 'A', 17.785, -21.22, 24.21], ['O', 0, 'A', 17.243, -20.629, 25.145], ['C', 0, 'A', 18.89, -23.341, 24.83], ['N', 0, 'A', 17.265, -21.283, 22.988], ['C', 0, 'A', 15.968, -20.714, 22.67], ['C', 0, 'A', 14.912, -21.814, 22.662], ['O', 0, 'A', 15.178, -22.955, 22.194], ['C', 0, 'A', 16.024, -20.074, 21.272], ['C', 0, 'A', 16.805, -18.754, 21.27], ['C', 0, 'A', 16.969, -18.176, 19.859], ['C', 0, 'A', 16.1, -17.707, 22.175], ['N', 0, 'A', 13.725, -21.522, 23.188], ['C', 0, 'A', 12.629, -22.463, 23.06], ['C', 0, 'A', 11.668, -22.0, 21.994], ['O', 0, 'A

In [55]:
g

Graph(num_nodes=3287, num_edges=166,
      ndata_schemes={}
      edata_schemes={'h': Scheme(shape=(1,), dtype=torch.float32)})

In [29]:
gg = dgl.DGLGraph()
gg.add_nodes(3)
gg.add_edges([1, 2, 0], [0, 1, 2])
gg.edata['h'] = th.tensor(np.zeros(3))
gg.ndata['h'] = th.tensor(np.zeros(3))
gg.ndata['h']

tensor([0., 0., 0.], dtype=torch.float64)

In [65]:
def func():
    return 1,2,3,4
a = func()
c, d, e, f = a
c, d, e

(1, 2, 3)